In [71]:
import math
import torch
from torch.nn.functional import conv2d

In [72]:
inputs = torch.randn(size = (2, 5, 5)) # （in_channel, H, W）
kernels = torch.randn(size = (3, 2, 3, 3)) # (out_channel, in_channel, kernel_size, kernel_size)
bias = torch.randn(3) #卷积偏置,输出通道数

In [73]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def matrix_conv2d(inputs, kernels, stride=1, padding=0, bias=None):
    """
    简单实现torch.nn.functional.conv2d
    """
    if padding:
        inputs = torch.nn.functional.pad(inputs, (padding, padding, padding, padding))
    in_channel, input_h, input_w = inputs.shape
    out_channel, _, kernel_h, kernel_w = kernels.shape
    
    output_h = math.floor((input_h - kernel_h) / stride) + 1 #计算输出特征图的shape
    output_w = math.floor((input_w - kernel_w) / stride) + 1
    
    output = torch.zeros(size=(out_channel, output_h, output_w)) # 初始化输出特征图
    
    for group in range(out_channel):
        group_kernel = kernels[group]
        out_ = torch.zeros(size=(in_channel, output_h, output_w)) #临时存放结果
        for item in range(in_channel):
            kernel = group_kernel[item]
            input_ = inputs[item]
            for i in range(0, input_h - kernel_h + 1, stride):
                for j in range(0, input_w - kernel_w + 1, stride):
                    region = input_[i:i+kernel_h, j:j+kernel_w]
                    out_[item,int(i/stride), int(j/stride)] = torch.sum(region * kernel) 
        if bias is not None:    
            output[group, :, :] = torch.sum(out_, dim=0) + bias[group]
        else:
            output[group, :, :] = torch.sum(out_, dim=0)
    
    return output

out = matrix_conv2d(inputs, kernels, bias=bias) # 手写卷积的输出

torch_api_input = inputs.unsqueeze(0)
torch_api_weight = kernels
torch_api_out = conv2d(torch_api_input, torch_api_weight, bias=bias) # torch api的输出
# 判断输出结果是否一致，小数点后3位一致即可
print(torch.allclose(out, torch_api_out, rtol=1e-3))
            
    

True
